### Grafana
##### Required Data & Folder List

In [ ]:
import botoplus.organization as _org

accounts = _org.accounts()

account_list = []

regions = []
regions.append('us-east-1')
regions.append('us-east-2')
regions.append('us-west-2')

for account in accounts:
    for region in regions:
        temp = {}
        temp['awsalias'] = account['awsalias']
        temp['awsaccount'] = str(account['awsaccount'])
        temp['name'] = account['awsalias']+' - '+str(account['awsaccount']+' - '+region)
        temp['region'] = region
        account_list.append(temp)
        print(temp)

print('Count: '+str(len(account_list)))

##### Cloud Watch Source List

In [ ]:
import requests
import botoplus.organization as _org

selected = _org.selected()

ssm = selected.client('ssm')
grafana_url = ssm.get_parameter(Name='/grafana/url', WithDecryption=False)['Parameter']['Value']
grafana_token = ssm.get_parameter(Name='/grafana/token', WithDecryption=True)['Parameter']['Value']

headers = {
    "Accept": "application/json",
    "Content-Type": "application/json",
    "Authorization": "Bearer "+grafana_token
}
url = grafana_url+'/api/datasources?query=json'
response = requests.get(url, headers=headers)
j = response.json()

data_sources = []

for source in j:
    
    if source['type'] == 'cloudwatch':
        temp = {}
        temp['uid'] = source['uid']
        temp['name'] = source['name']
        temp['region'] = source['jsonData']['defaultRegion']
        data_sources.append(temp)
        print(temp)

print('Count: '+str(len(data_sources)))

##### New Cloud Watch Sources

In [ ]:
import requests
import botoplus.organization as _org

selected = _org.selected()

ssm = selected.client('ssm')
grafana_url = ssm.get_parameter(Name='/grafana/url', WithDecryption=False)['Parameter']['Value']
grafana_token = ssm.get_parameter(Name='/grafana/token', WithDecryption=True)['Parameter']['Value']

headers = {
    "Accept": "application/json",
    "Content-Type": "application/json",
    "Authorization": "Bearer "+grafana_token
}
url = grafana_url+'/api/datasources?query=json'

for account in account_list:
    found = False
    for source in data_sources:
        if account['name'] == source['name']:
            found = True
            break
    if not found:

        j = {}
        j['orgId'] = 1
        j['name'] = account['name']
        j['type'] = 'cloudwatch'
        j['typeName'] = 'CloudWatch'
        j['typeLogoUrl'] = 'public/app/plugins/datasource/cloudwatch/img/amazon-web-services.png'
        j['access'] = 'proxy'
        j['basicAuth'] = False
        j['isDefault'] = False
        j['jsonData'] = {}
        j['jsonData']['assumeRoleArn'] = 'arn:aws:iam::'+str(account['awsaccount'])+':role/grafana'
        j['jsonData']['authType'] = 'grafana_assume_role'
        j['jsonData']['defaultRegion'] = account['region']

        response = requests.post(url, headers=headers, json=j)
        print(response.json())

##### Unused Cloud Watch Sources

In [ ]:
import requests
import botoplus.organization as _org

selected = _org.selected()

ssm = selected.client('ssm')
grafana_url = ssm.get_parameter(Name='/grafana/url', WithDecryption=False)['Parameter']['Value']
grafana_token = ssm.get_parameter(Name='/grafana/token', WithDecryption=True)['Parameter']['Value']

headers = {
    "Accept": "application/json",
    "Content-Type": "application/json",
    "Authorization": "Bearer "+grafana_token
}

for source in data_sources:
    found = False
    for account in account_list:
        if account['name'] == source['name']:
            found = True
            break
    if not found:
        url = grafana_url+'/api/datasources/uid/'+source['uid']
        response = requests.delete(url, headers=headers)
        print(response.json())

##### Existing Folder Lists

In [ ]:
import requests
import botoplus.organization as _org

selected = _org.selected()

ssm = selected.client('ssm')
grafana_url = ssm.get_parameter(Name='/grafana/url', WithDecryption=False)['Parameter']['Value']
grafana_token = ssm.get_parameter(Name='/grafana/token', WithDecryption=True)['Parameter']['Value']

headers = {
    "Accept": "application/json",
    "Content-Type": "application/json",
    "Authorization": "Bearer "+grafana_token
}
url = grafana_url+'/api/folders?query=json'
response = requests.get(url, headers=headers)
j = response.json()

existing_folder_names = []
folder_list = []

for folder in j:
    if folder['title'] != 'GrafanaCloud':
        existing_folder_names.append(folder['title'])
        parsed = folder['title'].split(' - ')
        if len(parsed) == 3:
            temp = {}
            temp['uid'] = folder['uid']
            temp['awsalias'] = parsed[0]
            temp['awsaccount'] = str(parsed[1])
            temp['region'] = parsed[2]
            folder_list.append(temp)
            print(temp)

print('Count: '+str(len(folder_list)))

##### New Folder Creation

In [ ]:
import requests
import botoplus.organization as _org

selected = _org.selected()

ssm = selected.client('ssm')
grafana_url = ssm.get_parameter(Name='/grafana/url', WithDecryption=False)['Parameter']['Value']
grafana_token = ssm.get_parameter(Name='/grafana/token', WithDecryption=True)['Parameter']['Value']

headers = {
    "Accept": "application/json",
    "Content-Type": "application/json",
    "Authorization": "Bearer "+grafana_token
}
url = grafana_url+'/api/folders?query=json'

for account in account_list:
    if account['name'] not in existing_folder_names:
        data = {
            "title": account['name']
        }
        r = requests.post(url, headers=headers, json=data)
        print(r.json())

##### Unused Folder Removal

In [ ]:
import requests
import botoplus.organization as _org

selected = _org.selected()

ssm = selected.client('ssm')
grafana_url = ssm.get_parameter(Name='/grafana/url', WithDecryption=False)['Parameter']['Value']
grafana_token = ssm.get_parameter(Name='/grafana/token', WithDecryption=True)['Parameter']['Value']

headers = {
    "Accept": "application/json",
    "Content-Type": "application/json",
    "Authorization": "Bearer "+grafana_token
}

for folder in folder_list:
    found = False
    for required in required_folder_list:
        if required['folder'] == folder['awsalias']+' - '+folder['awsaccount']:
            found = True
            break
    if not found:
        url = url = grafana_url+'/api/folders/'+folder['uid']+'?query=json'
        response = requests.delete(url, headers=headers)
        print(response.json())

##### Provisioning Format Export

In [ ]:
import json
import requests
import botoplus.organization as _org

selected = _org.selected()

ssm = selected.client('ssm')
grafana_url = ssm.get_parameter(Name='/grafana/url', WithDecryption=False)['Parameter']['Value']
grafana_token = ssm.get_parameter(Name='/grafana/token', WithDecryption=True)['Parameter']['Value']

headers = {
    "Accept": "application/json",
    "Content-Type": "application/json",
    "Authorization": "Bearer "+grafana_token
}
url = grafana_url+'/api/v1/provisioning/alert-rules/export?query=json'

response = requests.get(url, headers=headers)
j = response.json()

update_value = 'CHANGEIT'

for item in j['groups']:
    for rule in item['rules']:

        url = grafana_url+'/api/v1/provisioning/alert-rules/'+rule['uid']+'/export'
        response = requests.get(url, headers=headers)
        j = response.json()

        j = j['groups'][0]['rules'][0]
        j.pop('uid', None)
        j['orgID'] = 1
        j['data'][0]['model']['logGroups'] = []

        j['title'] = update_value
        j['folderUID'] = update_value
        j['ruleGroup'] = 'Alerts'
        j['data'][0]['datasourceUid'] = update_value
        j['data'][0]['model']['datasource']['uid'] = update_value
        j['data'][0]['model']['dimensions']['FunctionName'] = update_value
        j['data'][0]['model']['sql']['where']['expressions'][0]['operator']['value'] = update_value
        j['data'][0]['model']['sqlExpression'] = "SELECT SUM(Errors) FROM \"AWS/Lambda\" WHERE FunctionName = '"+update_value+"'"

        with open(update_value+'.json', 'w') as f:
            json.dump(j, f)

        print(json.dumps(j, indent=4))
        break
    break

##### Lambda Functions

In [ ]:
import botoplus.organization as _org

display_denied = False

accounts = _org.accounts()

lambda_functions = []

for account in accounts:
    print(str(account['awsaccount'])+' '+account['awsalias'])
    sessions = _org.sessions(account['awsaccount'])
    ec2 = sessions.client('ec2')
    regions = ec2.describe_regions()

    for region in regions['Regions']:
        try:
            lambda_region = sessions.client('lambda', region_name = region['RegionName'])
            paginator = lambda_region.get_paginator('list_functions')
            response_iterator = paginator.paginate()
            for page in response_iterator:
                if len(page['Functions']) > 0:
                    for item in page['Functions']:
                        temp = {}
                        temp['account'] = str(account['awsaccount'])
                        temp['alias'] = account['awsalias']
                        temp['name'] = item['FunctionName']
                        temp['region'] = region['RegionName']
                        lambda_functions.append(temp)
        except:
            if display_denied == True:
                print(' - '+region['RegionName']+' DENIED')
            pass

print('Count: '+str(len(lambda_functions)))

##### Alert List

In [ ]:
import requests
import botoplus.organization as _org

selected = _org.selected()

ssm = selected.client('ssm')
grafana_url = ssm.get_parameter(Name='/grafana/url', WithDecryption=False)['Parameter']['Value']
grafana_token = ssm.get_parameter(Name='/grafana/token', WithDecryption=True)['Parameter']['Value']

headers = {
    "Accept": "application/json",
    "Content-Type": "application/json",
    "Authorization": "Bearer "+grafana_token
}
url = grafana_url+'/api/v1/provisioning/alert-rules?query=json'

response = requests.get(url, headers=headers)
j = response.json()

alert_rules = []

for alert in j:
    temp = {}
    temp['uid'] = alert['uid']
    temp['title'] = alert['title']
    temp['ruleGroup'] = alert['ruleGroup']
    temp['folderUID'] = alert['folderUID']
    alert_rules.append(temp)
    print(temp)

print('Count: '+str(len(alert_rules)))

##### New Alert Creation

In [ ]:
import json
import requests
import botoplus.organization as _org

def alertuid(title, region, folderuid):
    for alert in alert_rules:
        if alert['title'] == title and alert['ruleGroup'] == region and alert['folderUID'] == folderuid:
            return alert['uid']
    return None

def folderuid(awsaccount, region):
    for folder in folder_list:
        if folder['awsaccount'] == awsaccount and folder['region'] == region:
            return folder['uid']
    return None

def sourceuid(awsaccount, region):
    for source in data_sources:
        parsed = source['name'].split(' - ')
        if parsed[1] == awsaccount and parsed[2] == region:
            return source['uid']
    return None

selected = _org.selected()

ssm = selected.client('ssm')
grafana_url = ssm.get_parameter(Name='/grafana/url', WithDecryption=False)['Parameter']['Value']
grafana_token = ssm.get_parameter(Name='/grafana/token', WithDecryption=True)['Parameter']['Value']

headers = {
    "Accept": "application/json",
    "Content-Type": "application/json",
    "Authorization": "Bearer "+grafana_token
}

for lambda_function in lambda_functions:

    source = sourceuid(str(lambda_function['account']), lambda_function['region'])
    folder = folderuid(str(lambda_function['account']), lambda_function['region'])
    notify = alertuid(lambda_function['name'], lambda_function['region'], folder)

    if notify == None and folder != None:

        with open('lambda-error-alert.json') as f:
            j = json.load(f)

        j['title'] = lambda_function['name']
        j['folderUID'] = folder
        j['data'][0]['datasourceUid'] = source
        j['data'][0]['model']['datasource']['uid'] = source
        j['data'][0]['model']['dimensions']['FunctionName'] = lambda_function['name']
        j['data'][0]['model']['sql']['where']['expressions'][0]['operator']['value'] = lambda_function['name']
        j['data'][0]['model']['sqlExpression'] = "SELECT SUM(Errors) FROM \"AWS/Lambda\" WHERE FunctionName = '"+lambda_function['name']+"'"

        if lambda_function['name'] == 'AmazonblocksStack-testAF53AC38-hB7qDOgAAMIK':

            url = grafana_url+'/api/v1/provisioning/alert-rules?query=json'
            response = requests.post(url, headers=headers, json=j)
            print(response.json())

            config = {}
            config['interval'] = 900
            url = grafana_url+'/api/v1/provisioning/folder/'+folder+'/rule-groups/Alerts?query=json'
            response = requests.put(url, headers=headers, json=config)

##### Unused Alert Removal

##### Delete Every Alert

In [ ]:
import requests
import botoplus.organization as _org

selected = _org.selected()

ssm = selected.client('ssm')
grafana_url = ssm.get_parameter(Name='/grafana/url', WithDecryption=False)['Parameter']['Value']
grafana_token = ssm.get_parameter(Name='/grafana/token', WithDecryption=True)['Parameter']['Value']

headers = {
    "Accept": "application/json",
    "Content-Type": "application/json",
    "Authorization": "Bearer "+grafana_token
}
url = grafana_url+'/api/v1/provisioning/alert-rules?query=json'

response = requests.get(url, headers=headers)
j = response.json()

alert_rules = []

for alert in j:
    url = grafana_url+'/api/v1/provisioning/alert-rules/'+alert['uid']+'?query=json'
    response = requests.delete(url, headers=headers)
    print(response.text)